In [ ]:
# CONSULTA PNCP — CONTRATAÇÕES (v1) /modulo-contratacoes/1_consultarContratacoes_PNCP_14133
# CONSULTA TODAS PAGINAS E TRAZ TODAS AS LICITAÇÕES DE UM PERIODO DE TEMPO DETERMINADO RETORNA UM GRANDE RESUMO DAS LICITAÇÕES
# A ABERTURA DE ITEM A ITEM SERÁ FEITO EM OUTRO SCRIPT 

import os
import time
import json
import pandas as pd
import requests

# ====== CONFIGURAÇÕES GERAIS ======
PASTA_DESTINO = r"C:\Users\VitorGiglio\Downloads\VG\Python\Dados\Licitações"
os.makedirs(PASTA_DESTINO, exist_ok=True)

URL = "https://dadosabertos.compras.gov.br/modulo-contratacoes/1_consultarContratacoes_PNCP_14133"
TOKEN = "1"  # substitua pelo token real, se tiver
HEADERS = {"Authorization": f"Bearer {TOKEN}", "accept": "application/json"}

# ====== PARÂMETROS OBRIGATÓRIOS ====== (ESCOLHA O PERÍODO DE TEMPO QUE QUER PEGAR) PENSEI EM PEGAR DIÁRIO 
DATA_INI = "2025-08-18"   # YYYY-MM-DD
DATA_FIM = "2025-08-22"   # YYYY-MM-DD
CODIGO_MODALIDADE = 6     # Ex.: 6 = Dispensa / 5 = Pregão (ajuste conforme sua necessidade)

# ====== PARÂMETROS OPCIONAIS (deixe "" para não filtrar) ======
FILTROS_OPCIONAIS = {
    "unidadeOrgaoCodigoUnidade": "",  # string
    "codigoOrgao": "",                # int
    "orgaoEntidadeCnpj": "",          # string
    "unidadeOrgaoCodigoIbge": "",     # string/int
    "unidadeOrgaoUfSigla": "",        # string
    "dataAtualizacaoPncp": "",        # YYYY-MM-DD
    "amparoLegalCodigoPncp": "",      # int
    # contratacaoExcluida é boolean; use True/False se quiser filtrar explicitamente
    # Ex.: "contratacaoExcluida": True
    "contratacaoExcluida": None
}

# ====== COLETA PAGINADA ======
tamanho_pagina = 490
pagina = 1
todos = []

while True:
    print(f"🔄 Página {pagina}...")

    params = {
        "pagina": pagina,
        "tamanhoPagina": tamanho_pagina,
        "dataPublicacaoPncpInicial": DATA_INI,
        "dataPublicacaoPncpFinal": DATA_FIM,
        "codigoModalidade": CODIGO_MODALIDADE,
    }

    # injeta só filtros opcionais que tenham valor (não vazio / não None)
    for k, v in FILTROS_OPCIONAIS.items():
        if v not in ("", None):
            params[k] = v

    r = requests.get(URL, headers=HEADERS, params=params)
    if r.status_code != 200:
        print(f"❌ Erro HTTP {r.status_code}: {r.text[:300]}")
        break

    payload = r.json()
    # Alguns endpoints retornam dict com "resultado"; outros, lista direta
    if isinstance(payload, dict):
        resultado = payload.get("resultado", [])
    else:
        resultado = payload

    if not resultado:
        print("✅ Fim dos dados.")
        break

    todos.extend(resultado)
    pagina += 1
    time.sleep(0.4)

# ====== SALVA JSON ======
nome_base = f"pncp_contratacoes_{DATA_INI}_a_{DATA_FIM}_mod{CODIGO_MODALIDADE}"
json_path = os.path.join(PASTA_DESTINO, f"{nome_base}.json")
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(todos, f, ensure_ascii=False, indent=2)
print(f"💾 JSON salvo em: {json_path}  ({len(todos)} registros)")

# ====== EXCEL ======
if todos:
    df = pd.DataFrame(todos)

    # Converte possíveis IDs longos para string (evita notação científica no Excel)
    provaveis_ids = [
        "idCompra", "idItemCompra", "cnpjFornecedor", "orgaoEntidadeCnpj",
        "uasg", "cpfVencedor", "numeroAviso", "numeroProcesso", "codigoPNCP",
        "idContrato", "idAditivo", "idItemContratacao"
    ]
    for col in provaveis_ids:
        if col in df.columns:
            df[col] = df[col].astype(str)

    excel_path = os.path.join(PASTA_DESTINO, f"{nome_base}.xlsx")
    df.to_excel(excel_path, index=False)
    print(f"📊 Excel salvo em: {excel_path}")
else:
    print("⚠️ Nenhum registro retornado para os filtros informados.")


🔄 Página 1...
🔄 Página 2...
🔄 Página 3...
🔄 Página 4...
🔄 Página 5...
🔄 Página 6...
🔄 Página 7...
✅ Fim dos dados.
💾 JSON salvo em: C:\Users\User\Downloads\Horom\Empresas\Horon Licitações\2 - Dados\2 - Licitações\pncp_contratacoes_2025-08-18_a_2025-08-22_mod6.json  (2899 registros)
📊 Excel salvo em: C:\Users\User\Downloads\Horom\Empresas\Horon Licitações\2 - Dados\2 - Licitações\pncp_contratacoes_2025-08-18_a_2025-08-22_mod6.xlsx
